In [1]:
import numpy as np
import pandas as pd
from os.path import join as pjoin

import bokeh

In [16]:
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_notebook, show, output_file, reset_output, gridplot, save

reset_output()
output_notebook()

Loading BokehJS ...

In [17]:
working_directory = "/home/alex/Documents/Projet_info/Deep-Reinforcement-Learning/callbacks"
n_actors = 8
n_plot = 5
list_files = ["rpe.csv", "rewards.csv", "random.csv", "diff.csv", "action.csv"]
list_titles = ["Reward per Environment", "Reward per Iteration", "Randomness","Norm of applied gradients", "gnActions taken"]
list_x_labels = ["Environments", "Iterations", "Iterations", "Iterations", "Iterations"]
list_y_labels = ["Reward", "Reward", "Random ?", "Norm", "Action"]
colors = ["orange", "lime", "navy", "darkred", "purple"]


In [23]:
reset_output()
output_notebook()

tabs = []
p = []
for j in range(n_actors):
    p.append([])
    for i in range(n_plot):
        data = pd.read_csv(pjoin(working_directory, "actor"+str(j), list_files[i]), header=None, names=["to_plot"])
        if i in [2, 4]:
            data["data"] = data["to_plot"].values
            data["to_plot"] = np.convolve(data.data, np.ones((100,))/100, mode='same')
        p[j].append(figure(title=list_titles[i], width=900, height=350))
        p[j][i].grid.grid_line_alpha = 0
        p[j][i].xaxis.axis_label = list_x_labels[i]
        p[j][i].yaxis.axis_label = list_y_labels[i]
        p[j][i].ygrid.band_fill_color = "olive"
        p[j][i].ygrid.band_fill_alpha = 0.1

        p[j][i].line(data.index, data["to_plot"], color=colors[i])
    tabs.append(Panel(child=column(p[j]), title="actor"+str(j)))

tabs_f = Tabs(tabs=tabs)
show(tabs_f)

Loading BokehJS ...